<a href="https://colab.research.google.com/github/BrenesRM/Haochen-Mitre-Attack-data-2024/blob/main/Haochen_Mitre_Attack_data_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Definition -

## Library Definition

In [3]:
!pip install numpy scipy matplotlib ipython scikit-learn pandas jupyter tensorflow tensorboard joblib mglearn \
attackcti stix2 nltk seaborn imblearn imbalanced-learn spacy flaml[automl] catboost lightgbm wordcloud

  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached mglearn-0.2.0-py2.py3-none-any.whl.metadata (628 bytes)
  Using cached attackcti-0.5.4-py3-none-any.whl.metadata (4.6 kB)
  Using cached stix2-3.0.1-py2.py3-none-any.whl.metadata (10 kB)
  Using cached imblearn-0.0-py2.py3-none-any.whl.metadata (355 bytes)
  Using cached catboost-1.2.8-cp311-cp311-manylinux2014_x86_64.whl.metadata (1.2 kB)
  Using cached FLAML-2.3.4-py3-none-any.whl.metadata (16 kB)
  Using cached jedi-0.19.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached jupyterlab-4.4.2-py3-none-any.whl.metadata (16 kB)
  Using cached taxii2_client-2.3.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached stix2_patterns-2.0.0-py2.py3-none-any.whl.metadata (8.3 kB)
  Using cached async_lru-2.0.5-py3-none-any.whl.metadata (4.5 kB)
  Using cached jupyter_lsp-2.2.5-py3-none-any.whl.metadata (1.8 kB)
  Using cached jupyter_server-2.16.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached jupyterlab_ser

In [4]:
from attackcti import attack_client
from stix2 import MemoryStore, AttackPattern, Identity, Relationship
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc, ConfusionMatrixDisplay
import seaborn as sns
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
import pickle

## Present the Data Get Alerts (scraped from open source)

In [8]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to your CSV files
base_path = '/content/drive/MyDrive/Colab Notebooks/Data/'

# Load the CSV files
data = pd.read_csv(base_path + 'tagged_alerts.csv')
data.dropna(inplace=True)

alerts = pd.read_csv(base_path + 'alerts.csv')
alerts.dropna(inplace=True)

# Display info
print("Amount of alerts:", len(alerts))
data.head()



Mounted at /content/drive
Amount of alerts: 2279


,id,alert
0,T1001,"This atomic test, named ""Steganographic Tarbal..."
1,T1001,This atomic test demonstrates the technique of...
2,T1003,The memory of lsass.exe is often dumped for of...
3,T1003,The memory of lsass.exe is often dumped for of...
4,T1003,The memory of lsass.exe is often dumped for of...


### Get the Label

Get All MITRE ATT&CK Techniques (id & description) from 'attackcti'






A Python module to access up to date ATT&CK content available in STIX via public TAXII server.

In [9]:
lift = attack_client()
all_techniques = lift.get_techniques()

An Example of MITRE ATT&CK (Label)

In [10]:
all_techniques[0]

AttackPattern(type='attack-pattern', spec_version='2.1', id='attack-pattern--0042a9f5-f053-4769-b3ef-9ad018dfa298', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2020-01-14T17:18:32.126Z', modified='2025-04-25T14:45:37.275Z', name='Extra Window Memory Injection', description="Adversaries may inject malicious code into process via Extra Window Memory (EWM) in order to evade process-based defenses as well as possibly elevate privileges. EWM injection is a method of executing arbitrary code in the address space of a separate live process. \n\nBefore creating a window, graphical Windows-based processes must prescribe to or register a windows class, which stipulate appearance and behavior (via windows procedures, which are functions that handle input/output of data).(Citation: Microsoft Window Classes) Registration of new windows classes can include a request for up to 40 bytes of EWM to be appended to the allocated memory of each instance of that class. This EWM

### Make the label understandable

In [11]:
def get_mitre_techniques():
    mitre_techniques_list = []
    for technique in all_techniques:
        if technique['external_references'] and technique['description']:
            for ref in technique['external_references']:
                if 'external_id' in ref and '.' not in ref['external_id']:
                    mitre_techniques_list.append((ref['external_id'], technique['name'],technique.description))
                    break
    return mitre_techniques_list
mitre_techniques = get_mitre_techniques()

mitre_techniques[0]

('T1047',
 'Windows Management Instrumentation',
 'Adversaries may abuse Windows Management Instrumentation (WMI) to execute malicious commands and payloads. WMI is designed for programmers and is the infrastructure for management data and operations on Windows systems.(Citation: WMI 1-3) WMI is an administration feature that provides a uniform environment to access Windows system components.\n\nThe WMI service enables both local and remote access, though the latter is facilitated by [Remote Services](https://attack.mitre.org/techniques/T1021) such as [Distributed Component Object Model](https://attack.mitre.org/techniques/T1021/003) and [Windows Remote Management](https://attack.mitre.org/techniques/T1021/006).(Citation: WMI 1-3) Remote WMI over DCOM operates using port 135, whereas WMI over WinRM operates over port 5985 when using HTTP and 5986 for HTTPS.(Citation: WMI 1-3) (Citation: Mandiant WMI)\n\nAn adversary can use WMI to interact with local and remote systems and use it as a 

Order the Array by TAG

In [14]:
mitre_id = [tech[0] for tech in mitre_techniques]
mitre_id

len(mitre_id)

392

### Get the Description of the TECHNIQUE

In [15]:
mitre_description = [tech[2] for tech in mitre_techniques]
mitre_description

['Adversaries may abuse Windows Management Instrumentation (WMI) to execute malicious commands and payloads. WMI is designed for programmers and is the infrastructure for management data and operations on Windows systems.(Citation: WMI 1-3) WMI is an administration feature that provides a uniform environment to access Windows system components.\n\nThe WMI service enables both local and remote access, though the latter is facilitated by [Remote Services](https://attack.mitre.org/techniques/T1021) such as [Distributed Component Object Model](https://attack.mitre.org/techniques/T1021/003) and [Windows Remote Management](https://attack.mitre.org/techniques/T1021/006).(Citation: WMI 1-3) Remote WMI over DCOM operates using port 135, whereas WMI over WinRM operates over port 5985 when using HTTP and 5986 for HTTPS.(Citation: WMI 1-3) (Citation: Mandiant WMI)\n\nAn adversary can use WMI to interact with local and remote systems and use it as a means to execute various behaviors, such as gathe